## Start

In [9]:
"""
Inference Script
"""
import json
import os
import torch
import tqdm
from shutil import copyfile


# Detectron imports|
from detectron2.engine import launch
from detectron2.data.dataset_mapper import DatasetMapper
from detectron2.data import *
from detectron2.data.build import trivial_batch_collator
from detectron2.data.samplers.distributed_sampler import InferenceSampler

# Project imports
import core.datasets.metadata as metadata

from core.setup import setup_config, setup_arg_parser
from offline_evaluation import compute_average_precision, compute_probabilistic_metrics, compute_calibration_errors
from probabilistic_inference.probabilistic_inference import build_predictor
from probabilistic_inference.inference_utils import instances_to_json

from baal.active import FileDataset, ActiveLearningDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [10]:
##setup inference args, this also contains all the training args
arg_parser = setup_arg_parser()
args = arg_parser.parse_args("")
# Support single gpu inference only.
args.num_gpus = 1
args.dataset_dir = '/public-dataset/BDD/bdd100k'
args.test_dataset = 'bdd_val'
args.config_file = '/home/richard.tanai/cvpr2/pod_compare/src/configs/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout.yaml'
args.inference_config = '/home/richard.tanai/cvpr2/pod_compare/src/configs/Inference/bayes_od_mc_dropout.yaml'
print("Command Line Args:", args)

Command Line Args: Namespace(config_file='/home/richard.tanai/cvpr2/pod_compare/src/configs/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout.yaml', dataset_dir='/public-dataset/BDD/bdd100k', dist_url='tcp://127.0.0.1:50162', eval_only=False, inference_config='/home/richard.tanai/cvpr2/pod_compare/src/configs/Inference/bayes_od_mc_dropout.yaml', iou_correct=0.7, iou_min=0.1, machine_rank=0, min_allowed_score=0.0, num_gpus=1, num_machines=1, opts=[], random_seed=0, resume=False, test_dataset='bdd_val')


In [11]:
global result_list

In [12]:
#method inside the build detection train loader that loads from config
def load_dataset_and_mapper_from_config(cfg, dataset_name='bdd_val', mapper=None):
    """
    Uses the given `dataset_name` argument (instead of the names in cfg), because the
    standard practice is to evaluate each test set individually (not combining them).
    """
    dataset = get_detection_dataset_dicts(
        [dataset_name],
        filter_empty=False,
        proposal_files=[
            cfg.DATASETS.PROPOSAL_FILES_TEST[list(cfg.DATASETS.TEST).index(dataset_name)]
        ]
        if cfg.MODEL.LOAD_PROPOSALS
        else None,
    )
    # use active learning dataset wrapper
    if mapper is None:
        mapper = DatasetMapper(cfg, False)
        
    return dataset, mapper
    
    

In [13]:
def dataloader_from_config(cfg, dataset, mapper):
    if isinstance(dataset, list):
        dataset = DatasetFromList(dataset, copy=False)
    if mapper is not None:
        dataset = MapDataset(dataset, mapper)
    sampler = InferenceSampler(len(dataset))
    # Always use 1 image per worker during inference since this is the
    # standard when reporting inference time in papers.
    batch_sampler = torch.utils.data.sampler.BatchSampler(sampler, 1, drop_last=False)
    num_worker = 4
    data_loader = torch.utils.data.DataLoader(
        dataset,
        num_workers=num_worker,
        batch_sampler=batch_sampler,
        collate_fn=trivial_batch_collator,
    )
    return data_loader

In [14]:
#method inside the build detection train loader that loads from config
def test_loader_from_config(cfg, dataset_name='bdd_val', mapper=None):
    """
    Uses the given `dataset_name` argument (instead of the names in cfg), because the
    standard practice is to evaluate each test set individually (not combining them).
    """
    dataset = get_detection_dataset_dicts(
        [dataset_name],
        filter_empty=False,
        proposal_files=[
            cfg.DATASETS.PROPOSAL_FILES_TEST[list(cfg.DATASETS.TEST).index(dataset_name)]
        ]
        if cfg.MODEL.LOAD_PROPOSALS
        else None,
    )
    # use active learning dataset wrapper
    dataset = ActiveLearningDataset(dataset)
    dataset.label_randomly(100)
    if mapper is None:
        mapper = DatasetMapper(cfg, False)
    
    if isinstance(dataset, list):
        dataset = DatasetFromList(dataset, copy=False)
    if mapper is not None:
        dataset = MapDataset(dataset, mapper)
    sampler = InferenceSampler(len(dataset))
    # Always use 1 image per worker during inference since this is the
    # standard when reporting inference time in papers.
    batch_sampler = torch.utils.data.sampler.BatchSampler(sampler, 1, drop_last=False)
    num_worker = 4
    data_loader = torch.utils.data.DataLoader(
        dataset,
        num_workers=num_worker,
        batch_sampler=batch_sampler,
        collate_fn=trivial_batch_collator,
    )
    return data_loader

In [15]:
# run this once per session only
cfg = setup_config(args, random_seed=args.random_seed, is_testing=True)

WARNING [02/05 17:25:53 fvcore.common.config]: Loading config /home/richard.tanai/cvpr2/pod_compare/src/configs/BDD-Detection/retinanet/../../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
WARNING [02/05 17:25:53 d2.config.compat]: Config '/home/richard.tanai/cvpr2/pod_compare/src/configs/Inference/bayes_od_mc_dropout.yaml' has no VERSION. Assuming it to be compatible with latest v2.
[02/05 17:25:53 detectron2]: Rank of current process: 0. World size: 1
[02/05 17:25:54 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------------
sys.platform            linux
Python                  3.8.5 (default, Sep  4 2020, 07:30:14) [GCC 7.3.0]
numpy                   1.19.5
detectron2              0.3 @/opt/anaconda3/envs/pod/lib/python3.8/site-packages/detectron2
Compiler                GCC 7.3
CUDA compiler           CUDA 10.2
detectron2 arch flags   3.7, 5.0, 5.2, 6.0

[02/05 17:25:54 detectron2]: Full config saved to /home/richard.tanai/cvpr2/pod_compare/data/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout/random_seed_0/config.yaml


AssertionError: Dataset 'bdd_train' is already registered!

In [16]:
#cfg = setup_config(args, random_seed=args.random_seed, is_testing=True)

# Make sure only 1 data point is processed at a time. This simulates
# deployment.
cfg.defrost()
cfg.DATALOADER.NUM_WORKERS = 32
cfg.SOLVER.IMS_PER_BATCH = 1

cfg.MODEL.DEVICE = device.type

# Set up number of cpu threads
torch.set_num_threads(cfg.DATALOADER.NUM_WORKERS)

# Create inference output directory and copy inference config file to keep
# track of experimental settings
inference_output_dir = os.path.join(
    cfg['OUTPUT_DIR'],
    'inference',
    args.test_dataset,
    os.path.split(args.inference_config)[-1][:-5])
os.makedirs(inference_output_dir, exist_ok=True)
copyfile(args.inference_config, os.path.join(
    inference_output_dir, os.path.split(args.inference_config)[-1]))

# Get category mapping dictionary:
train_thing_dataset_id_to_contiguous_id = MetadataCatalog.get(
    cfg.DATASETS.TRAIN[0]).thing_dataset_id_to_contiguous_id
test_thing_dataset_id_to_contiguous_id = MetadataCatalog.get(
    args.test_dataset).thing_dataset_id_to_contiguous_id

# If both dicts are equal or if we are performing out of distribution
# detection, just flip the test dict.
if (train_thing_dataset_id_to_contiguous_id == test_thing_dataset_id_to_contiguous_id) or (
        cfg.DATASETS.TRAIN[0] == 'coco_not_in_voc_2017_train'):
    cat_mapping_dict = dict(
        (v, k) for k, v in test_thing_dataset_id_to_contiguous_id.items())
else:
    # If not equal, two situations: 1) BDD to KITTI and 2) COCO to PASCAL
    cat_mapping_dict = dict(
        (v, k) for k, v in test_thing_dataset_id_to_contiguous_id.items())
    if 'voc' in args.test_dataset and 'coco' in cfg.DATASETS.TRAIN[0]:
        dataset_mapping_dict = dict(
            (v, k) for k, v in metadata.COCO_TO_VOC_CONTIGUOUS_ID.items())
    elif 'kitti' in args.test_dataset and 'bdd' in cfg.DATASETS.TRAIN[0]:
        dataset_mapping_dict = dict(
            (v, k) for k, v in metadata.BDD_TO_KITTI_CONTIGUOUS_ID.items())
    else:
        ValueError(
            'Cannot generate category mapping dictionary. Please check if training and inference datasets are compatible.')
    cat_mapping_dict = dict(
        (dataset_mapping_dict[k], v) for k, v in cat_mapping_dict.items())

# Build predictor
predictor = build_predictor(cfg)
#orig_test_data_loader = build_detection_test_loader(cfg, dataset_name=args.test_dataset)
#test_data_loader = test_loader_from_config(cfg, dataset_name='bdd_val', mapper=None)

dataset, mapper = load_dataset_and_mapper_from_config(cfg, dataset_name='bdd_val', mapper=None)

dataset = ActiveLearningDataset(dataset)
dataset.label([1,2,3])
#dataset.label_randomly(100)

test_data_loader = dataloader_from_config(cfg, dataset, mapper)

final_output_list = []
if not args.eval_only:
    with torch.no_grad():
        with tqdm.tqdm(total=len(test_data_loader)) as pbar:
            for idx, input_im in enumerate(test_data_loader):
                #print(input_im.size)
                outputs = predictor(input_im)

                final_output_list.extend(
                    instances_to_json(
                        outputs,
                        input_im[0]['image_id'],
                        cat_mapping_dict))
                pbar.update(1)
                result_list = outputs
                #break

    with open(os.path.join(inference_output_dir, 'coco_instances_results.json'), 'w') as fp:
        json.dump(final_output_list, fp, indent=4,
                  separators=(',', ': '))

compute_average_precision.main(args, cfg)
#compute_probabilistic_metrics.main(args, cfg)
#compute_calibration_errors.main(args, cfg)

[02/05 17:25:57 d2.data.datasets.coco]: Loaded 10000 images in COCO format from /public-dataset/BDD/bdd100k/labels/val_coco_format.json
[02/05 17:25:58 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]


  0%|          | 0/3 [00:00<?, ?it/s]


RuntimeError: cannot perform reduction function max on tensor with no elements because the operation does not have an identity

In [ ]:
outputs

In [ ]:
dataset[4]

In [ ]:
len(dataset.pool)

In [ ]:
dataset.label([0])

In [ ]:
dataset.pool[0]

In [ ]:
dataset.pool[0]

In [ ]:
data

In [ ]:
orig_test_data_loader = build_detection_test_loader(cfg, dataset_name=args.test_dataset)

In [ ]:
orig_test_data_loader

In [ ]:
test_data_loader

In [ ]:
result_list.pred_boxes_covariance.size()

In [ ]:
len(result_list)

In [ ]:
res_img = predictor.visualize_inference(input_im, outputs)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(res_img)
plt.show

In [ ]:
next(iter(test_data_loader))[0]

In [ ]:
raw_image = next(iter(test_data_loader))[0]['image']
plt.imshow(raw_image)
plt.show